# Dataset preparation


### 1. Load libraries

In [ ]:
import pandas as pd
import numpy as np

### 2. Prepare consumption data

In [ ]:
# Input the folder where your data is located here:
data_source_folder = ''

# Load csv
df = pd.read_csv("{}/consumption.csv".format(data_source_folder), sep=";", decimal = ',')

# Create date-time variable
df['date'] = pd.date_range(start = '01/01/2018 00:00:00', periods = 104178, freq = '15T')

In [ ]:
# Prepare data
df_consumption = df[['Datum','Ist', 'date']]
df_consumption['Ist'] = df_consumption['Ist'].astype(float)

# subset to Januray 2018 - March 2019
df_mask = (df_consumption['date'] < '2019/04/01') & (df_consumption['date']>='2018/01/01')
df_consumption = df_consumption.loc[df_mask]

df_consumption.set_index('date', inplace=True)
df_consumption = df_consumption[['Ist']]
df_consumption.rename(columns={"Ist": "consumption"}, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# turn consumption into MWh
df_consumption['consumption'] = df_consumption['consumption']/1000

In [ ]:
df_consumption[df_consumption['consumption'] < 10] = np.nan
df_consumption.fillna( method = 'ffill', inplace = True)

#### Save consumption dataset to csv

In [ ]:
df_consumption.to_csv("{}/consumption_prepared.csv".format(data_source_folder))

### 3. Weather data

Naming of relevant data columns:
- FM = daily average wind speed (km/h)
- NM = daily average cloud cover (%)
- TMK = daily average temperature (°C)

In [ ]:
#Import raw datasets
df_weather_saarbrucken = pd.read_csv("{}/daily_weather_saarbrücken.csv".format(data_source_folder), sep=";", decimal = '.')
df_weather_trier = pd.read_csv("{}/daily_weather_trier.csv".format(data_source_folder), sep=",", decimal = '.')
df_weather_dusseldorf = pd.read_csv("{}/daily_weather_dus.csv".format(data_source_folder), sep=",", decimal = '.')

In [ ]:
# Change column names
df_weather_trier = df_weather_trier[['MESS_DATUM', '  FM', '  NM', ' TMK']]
df_weather_trier[df_weather_trier < 0] = np.nan # mark missing values as Nan
df_weather_trier = df_weather_trier.iloc[28000:, :]

df_weather_dusseldorf = df_weather_dusseldorf[['MESS_DATUM', '  FM', '  NM', ' TMK']]
df_weather_dusseldorf[df_weather_dusseldorf < 0] = np.nan # mark missing values as Nan

df_weather_saarbrucken = df_weather_saarbrucken[['MESS_DATUM', '  FM', '  NM', ' TMK']]
df_weather_saarbrucken[df_weather_saarbrucken < 0] = np.nan # mark missing values as Nan

In [ ]:
# Compute date-time information/subset to relevant time frame

# weather trier
df_weather_dusseldorf['date'] = pd.date_range(start = '01/01/1952', periods = 24837, freq = 'D')
df_mask = (df_weather_dusseldorf['date'] > '2017/12/30') & (df_weather_dusseldorf['date'] < '2019/04/01')
df_weather_dusseldorf = df_weather_dusseldorf.loc[df_mask]
df_weather_dusseldorf.set_index('date', inplace=True)
df_weather_dusseldorf.columns = ['date_old', 'wind', 'cloud', 'temp']
df_weather_dusseldorf = df_weather_dusseldorf[['temp', 'wind', 'cloud']]
df_weather_dusseldorf.fillna( method = 'ffill', inplace = True)

# weather trier
df_weather_trier['date'] = pd.date_range(start = '08/30/2017', periods = 854, freq = 'D')
df_mask = (df_weather_trier['date']>'2017/12/30') & (df_weather_trier['date'] < '2019/04/01')
df_weather_trier = df_weather_trier.loc[df_mask]
df_weather_trier.set_index('date', inplace=True)
df_weather_trier.columns = ['date_old', 'wind', 'cloud', 'temp']
df_weather_trier = df_weather_trier[['temp', 'wind', 'cloud']]
df_weather_trier.fillna( method = 'ffill', inplace = True)


# Weather saarbrucken
df_weather_saarbrucken['date'] = pd.date_range(start = '01/01/1951', periods = 25202, freq = 'D')
df_mask = (df_weather_saarbrucken['date']>'2017/12/30') & (df_weather_saarbrucken['date'] < '2019/04/01')
df_weather_saarbrucken = df_weather_saarbrucken.loc[df_mask]
df_weather_saarbrucken.set_index('date', inplace=True)
df_weather_saarbrucken.columns = ['date_old', 'wind', 'cloud', 'temp']
df_weather_saarbrucken = df_weather_saarbrucken[['temp', 'wind', 'cloud']]
df_weather_saarbrucken.fillna( method = 'ffill', inplace = True)

### 4. Combine datasets to average

In [ ]:
# Combine all weather records into one dataframe
daily_weather_avg = pd.DataFrame()

daily_weather_avg['temp'] = (df_weather_dusseldorf['temp']+df_weather_trier['temp']+df_weather_saarbrucken['temp'])/3
daily_weather_avg['wind'] = (df_weather_dusseldorf['wind']+df_weather_trier['wind']+df_weather_saarbrucken['wind'])/3
daily_weather_avg['cloud'] = (df_weather_dusseldorf['cloud']+df_weather_trier['cloud']+df_weather_saarbrucken['cloud'])/3

In [ ]:
# Transform cloud into % (currently in x/8)
daily_weather_avg['cloud'] = daily_weather_avg['cloud']/8

### 5. Save combined dataset to csv

In [ ]:
daily_weather_avg.to_csv("{}/daily_weather_avg.csv".format(data_source_folder))